In [72]:
import csv
import string
import re
from sklearn import svm
import sklearn
import gzip
import random
import itertools
from collections import defaultdict
import numpy as np
import datetime

#from sklearn.metrics import jaccard_similarity_score
#from scipy.sparse import csc_matrix

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.linear_model import LogisticRegression

In [475]:
#players = []
#heroNames = []
#matches = []

data = []


with open('match.csv') as csvfile:
    keys = list(csvfile.readline().split(',')) #note: last element has a newline
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['match_id', 'duration', 'radiant_win']
    types = [int, int, lambda x: x == "True"]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        datum['players'] = []
        datum['chat'] = []
        datum['objectives'] = []
        datum['times'] = []
        data.append(datum)


with open('players.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['hero_id', 'gold_spent']
    types = [int, int]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        data[int(row[0])]['players'].append(datum)

        
with open('chat.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['key', 'slot']
    types = [str, int]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        data[int(row[0])]['chat'].append(datum)


with open('objectives.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    spamreader = csv.reader(csvfile, delimiter=',')
    desiredKeys = ['key', 'player1', 'subtype', 'time']
    types = [lambda x: 34 if x == "" else int(float(x)), int, str, int]
    desiredKeysIndices = [keys.index(x) for x in desiredKeys]
    for row in spamreader:
        datum = {}
        for i in range(len(desiredKeys)):
            datum[desiredKeys[i]] = types[i](row[desiredKeysIndices[i]])
        data[int(row[0])]['objectives'].append(datum)


        
        
with open('player_time.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    keys[-1] = keys[-1][0:-1]
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        data[int(row[0])]['times'].append(row[2:])

        



In [499]:
heroToId = {}
idToHero = {}

with open('hero_names.csv') as csvfile:
    keys = list(csvfile.readline().split(','))
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        heroToId[row[2]] = int(row[1])
        idToHero[int(row[1])] = row[2]
heroToId['Monkey King'] = 0
idToHero[0] = 'Monkey King'

In [500]:
random.shuffle(data)
train = data[:40000]
val = data[40000:]

In [456]:
advantage = {}

smd = open('michael.csv')

for line in smd:
    shit = {}
    dudes = list(line.split(', '))
    dudes[2] = float(dudes[2][:4])
    
    if dudes[0] not in advantage:
        shit[dudes[1]] = dudes[2]
        advantage[dudes[0]] = shit
    else:
        advantage[dudes[0]][dudes[1]] = dudes[2]

In [516]:
for key1 in advantage:
    advantage[key1][key1] = 0.0

In [518]:
def advantage_baseline(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
        

        feat1 = [0]*10
        ind = 0
        
        for p1 in d['players'][:5]:
            for p2 in d['players'][5:]:
                feat1[ind] = feat1[ind] + advantage[idToHero[p1['hero_id']]][idToHero[p2['hero_id']]]
            ind = ind + 1
           
        for p1 in d['players'][5:]:
            for p2 in d['players'][:5]:     
                feat1[ind] = feat1[ind] + advantage[idToHero[p1['hero_id']]][idToHero[p2['hero_id']]]
            ind = ind + 1
    
        return feat + feat1
    
    
    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [502]:
advantage['Monkey King']

{'Abaddon': 1.19,
 'Alchemist': 0.47,
 'Ancient Apparition': 0.02,
 'Anti-Mage': 2.4,
 'Arc Warden': -0.1,
 'Axe': 1.23,
 'Bane': -0.0,
 'Batrider': -2.3,
 'Beastmaster': -0.2,
 'Bloodseeker': 1.56,
 'Bounty Hunter': -0.2,
 'Brewmaster': 0.11,
 'Bristleback': -0.4,
 'Broodmother': -0.8,
 'Centaur Warrunner': 1.43,
 'Chaos Knight': 0.3,
 'Chen': -1.0,
 'Clinkz': -0.6,
 'Clockwerk': 0.47,
 'Crystal Maiden': 1.1,
 'Dark Seer': 0.3,
 'Dazzle': 0.67,
 'Death Prophet': 0.82,
 'Disruptor': 0.24,
 'Doom': 0.22,
 'Dragon Knight': 0.67,
 'Drow Ranger': 1.67,
 'Earth Spirit': -0.7,
 'Earthshaker': 0.92,
 'Elder Titan': 0.63,
 'Ember Spirit': 1.25,
 'Enchantress': -1.0,
 'Enigma': 0.21,
 'Faceless Void': 1.32,
 'Gyrocopter': -0.1,
 'Huskar': 1.27,
 'Invoker': 0.65,
 'Io': -1.7,
 'Jakiro': 0.39,
 'Juggernaut': 0.67,
 'Keeper of the Light': -1.0,
 'Kunkka': 0.71,
 'Legion Commander': 1.21,
 'Leshrac': 0.05,
 'Lich': -0.0,
 'Lifestealer': 0.49,
 'Lina': -0.2,
 'Lion': -0.4,
 'Lone Druid': -1.6,
 'Lun

In [407]:
def accuracy(val_lbls, pred):
    numright = 0
    for i in range(len(val_lbls)):
        if val_lbls[i] == pred[i]:
            numright += 1
    return float(numright) / len(val_lbls)

def classify(tf, tl, vf, vl):
    classifiers = [
    #KNeighborsClassifier(3),
    svm.LinearSVC(C=1),
    #svm.LinearSVC(gamma=2, C=1),
    #GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=4),
    RandomForestClassifier(max_depth=4, n_estimators=50, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    #QuadraticDiscriminantAnalysis()
    LogisticRegression()
    ]
    
    for clf in classifiers:
        clf.fit(tf, tl)
        pred = clf.predict(vf)
        print accuracy(vl, pred)


def baseline(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
        return feat

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [533]:
def everything(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        #msgs per team
        feat1 = [0, 0]
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                feat1[0] += 1
            else:
                feat1[1] += 1
        
        #heroes
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
            
        #toxic words per team
        feat2 = [0,0]
        str1 = ""
        str2 = ""
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                str1 += msg['key']
            else:
                str2 += msg['key']
        str1 = str1.lower()
        str2 = str2.lower()
        for word in profanity:
            feat2[0] += str1.count(word)
            feat2[1] += str2.count(word)
        
        #words per team
        feat3 = [0,0]
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                feat3[0] += len(msg['key'].split(" "))
            else:
                feat3[1] += len(msg['key'].split(" "))

        
        #early objectives
        feat4 = [0, 0]
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_FIRSTBLOOD':
                if o['player1'] < 4.5:
                    feat4[0] = 1
                else:
                    feat4[0] = -1
                break
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_TOWER_KILL':
                if o['player1'] < 4.5:
                    feat4[1] = 1
                else:
                    feat4[1] = -1
                break
    
        
        #advantage
        feat5 = [0]*10
        ind = 0
        
        for p1 in d['players'][:5]:
            for p2 in d['players'][5:]:
                feat5[ind] = feat5[ind] + advantage[idToHero[p1['hero_id']]][idToHero[p2['hero_id']]]
            ind = ind + 1
           
        for p1 in d['players'][5:]:
            for p2 in d['players'][:5]:     
                feat5[ind] = feat5[ind] + advantage[idToHero[p1['hero_id']]][idToHero[p2['hero_id']]]
            ind = ind + 1
    
    
        #first 5 minutes
        feat6 = [0, 0, 0, 0, 0, 0]
        minute = 5
        if len(d['times']) > minute:
            row = d['times'][minute]
            row = [int(x) for x in row]
            for i in range(15):
                if i % 3 == 0:
                    feat6[0] += row[i]
                elif i % 3 == 1:
                    feat6[1] += row[i]
                elif i % 3 == 2:
                    feat6[2] += row[i]
            for i in range(15, 30):
                if i % 3 == 0:
                    feat6[3] += row[i]
                elif i % 3 == 1:
                    feat6[4] += row[i]
                elif i % 3 == 2:
                    feat6[5] += row[i]    
    
        return feat + feat1 + feat2 + feat3 + feat4 + feat5 + feat6
    
    
    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [527]:
def chat_based(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        #msgs per team
        feat1 = [0, 0]
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                feat1[0] += 1
            else:
                feat1[1] += 1
        
        #heroes
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
            
        #toxic words per team
        feat2 = [0,0]
        str1 = ""
        str2 = ""
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                str1 += msg['key']
            else:
                str2 += msg['key']
        str1 = str1.lower()
        str2 = str2.lower()
        for word in profanity:
            feat2[0] += str1.count(word)
            feat2[1] += str2.count(word)
        
        #words per team
        feat3 = [0,0]
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                feat3[0] += len(msg['key'].split(" "))
            else:
                feat3[1] += len(msg['key'].split(" "))

        return feat + feat1 + feat2 + feat3
    
    
    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [410]:
chat_based(train, val)

0.6487
0.5913
0.5143
0.6757
0.643
0.5818
0.6562


In [522]:
everything(train, val) #with advantage

0.6419
0.615
0.5196
0.6869
0.6728
0.6023
0.6907


In [524]:
everything(train, val) #no advantage

0.6664
0.615
0.5209
0.7001
0.6753
0.6019
0.6865


In [532]:
everything(train, val) #with first 5 minutes no advantage

0.4834
0.6117
0.5225
0.5232
0.6849
0.617
0.7061


In [534]:
everything(train, val) #with first 5 minutes with advantage

0.5908
0.6117
0.5244
0.5529
0.6838
0.6169
0.7096


In [218]:
profanity = []
with open('profanity.txt') as txtfile:
    for item in txtfile:
        profanity.append(item[0:-1].lower())

In [219]:
def toxic(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    def feature(d):
        feat = [0, 0]
        str1 = ""
        str2 = ""
        for msg in d['chat']:
            if msg['slot'] < 4.5:
                str1 += msg['key']
            else:
                str2 += msg['key']
        str1 = str1.lower()
        str2 = str2.lower()
        for word in profanity:
            feat[0] += str1.count(word)
            feat[1] += str2.count(word)
        return feat

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [220]:
toxic(train, val)

0.5592
radiant more toxic
False
0.5591
radiant more toxic
False
0.5597
radiant more toxic
False
0.5612
radiant more toxic
False
0.5606
radiant more toxic
False
0.5472
radiant more toxic
False
0.5592
radiant more toxic
False


In [320]:
def getAverageHeroPositions(train):
    heroIdToPosition = {}
    totals = {}
    
    for d in train:
        
        players1 = d['players'][:5]
        players2 = d['players'][5:]
        players1 = sorted([(x['gold_spent'], x['hero_id']) for x in players1])
        players2 = sorted([(x['gold_spent'], x['hero_id']) for x in players2])
        

        for i in range(len(players1)):
            heroId = players1[i][1]
            if heroId not in heroIdToPosition:
                heroIdToPosition[heroId] = 0
                totals[heroId] = 0
            heroIdToPosition[heroId] += i
            totals[heroId] += 1

        for i in range(len(players2)):
            heroId = players2[i][1]
            if heroId not in heroIdToPosition:
                heroIdToPosition[heroId] = 0
                totals[heroId] = 0
            heroIdToPosition[heroId] += i
            totals[heroId] += 1

    for heroId in heroIdToPosition.keys():
        heroIdToPosition[heroId] = float(heroIdToPosition[heroId]) / totals[heroId]

    return heroIdToPosition
        
    

In [321]:
heroIdToPosition = getAverageHeroPositions(train)

In [344]:
def synergy(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    
    
    
    bestTeam = 0
    for d in train:
        summ = 0.0
        if d['radiant_win']:
            for p in d['players'][:5]:
                summ += heroIdToPosition[p['hero_id']]
        else:
            for p in d['players'][5:]:
                summ += heroIdToPosition[p['hero_id']]
        summ /= 5
        bestTeam += summ
    bestTeam /= len(train)
    
    def feature(d):
        feat1 = []
        feat2 = []
        for p in d['players'][:5]:
            #feat1[0] += heroIdToPosition[p['hero_id']]
            feat1.append(heroIdToPosition[p['hero_id']])
        for p in d['players'][5:]:
            #feat2[0] += heroIdToPosition[p['hero_id']]
            feat2.append(heroIdToPosition[p['hero_id']])

        
        feat = [0]*226
        for p in d['players'][:5]:
            feat[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat[p['hero_id'] + 113] = 1
        

        #feat1[0] = abs(feat1[0] - 10.0)
        #feat2[0] = abs(feat2[0] - 10.0)
        return sorted(feat1) + sorted(feat2)

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [332]:
baseline(train, val)

0.6023
0.5204
0.5143
0.5924
0.5874
0.5832
0.6023


In [345]:
synergy(train, val)

0.5422
0.535
0.5428
0.5397
0.5536
0.5419
0.5419


In [454]:
def time_based(train, val):
    train_feats = []
    train_lbls = []
    val_feats = []
    val_lbls = []
    

    def feature(d):
        feat = [0, 0, 0, 0, 0]
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_FIRSTBLOOD':
                if o['player1'] < 4.5:
                    feat[0] = 1
                else:
                    feat[0] = -1
                break
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_TOWER_KILL':
                if o['player1'] < 4.5:
                    feat[1] = 1
                else:
                    feat[1] = -1
                break
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_ROSHAN_KILL':
                if o['player1'] < 4.5:
                    feat[2] = 1
                else:
                    feat[2] = -1
                break
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_AEGIS':
                if o['player1'] < 4.5:
                    feat[3] = 1
                else:
                    feat[3] = -1
                break
        for o in d['objectives']:
            if o['subtype'] == 'CHAT_MESSAGE_BARRACKS_KILL':
                if o['key'] < 34:
                    feat[4] = 1
                elif o['key'] > 34:
                    feat[4] = -1
                break


        feat1 = [0]*226
        for p in d['players'][:5]:
            feat1[p['hero_id']] = 1
        for p in d['players'][5:]:
            feat1[p['hero_id'] + 113] = 1

            
            
            
        feat2 = [0, 0, 0]
        feat3 = [0, 0, 0]
        minute = 5
        if len(d['times']) <= minute:
            print "hi"
            return feat2 #+ feat3
        row = d['times'][minute]
        row = [int(x) for x in row]
        for i in range(15):
            if i % 3 == 0:
                feat2[0] += row[i]
            elif i % 3 == 1:
                feat2[1] += row[i]
            elif i % 3 == 2:
                feat2[2] += row[i]
        for i in range(15, 30):
            if i % 3 == 0:
                feat3[0] += row[i]
            elif i % 3 == 1:
                feat3[1] += row[i]
            elif i % 3 == 2:
                feat3[2] += row[i]
        
        feat4 = [(feat2[0]-feat3[0])**2 if feat2[0] > feat3[0] else -(feat2[0]-feat3[0])**2,
                 (feat2[1]-feat3[1])**2 if feat2[1] > feat3[1] else -(feat2[1]-feat3[1])**2,
                 (feat2[2]-feat3[2])**2 if feat2[2] > feat3[2] else -(feat2[2]-feat3[2])**2]
        
        
        return feat4

    for d in train:
        train_feats.append(feature(d))
        train_lbls.append(d['radiant_win'])
    
    for d in val:
        val_feats.append(feature(d))
        val_lbls.append(d['radiant_win'])
    
    classify(train_feats, train_lbls, val_feats, val_lbls)

In [455]:
time_based(train, val)

hi
hi
hi
hi
0.5951
0.6074
0.6061
0.5547
0.6067
0.5981
0.6059


In [520]:
baseline(train, val)

0.6031
0.5377
0.5195
0.6064
0.5903
0.5872
0.6028


In [519]:
advantage_baseline(train, val)

0.6152
0.5372
0.5199
0.6079
0.5895
0.5873
0.613
